In [4]:
import ANNModel as ann
import os
import cv2
import numpy as np

In [5]:
def load_mnist_dataset(dataset, path):
        labels = os.listdir(os.path.join(path, dataset))
        X = []
        y = []
        for label in labels:
            for file in os.listdir(os.path.join(path, dataset, label)):
                image = cv2.imread(os.path.join(path, dataset, label, file),cv2.IMREAD_UNCHANGED)
                X.append(image)
                y.append(label)
        return np.array(X), np.array(y).astype('uint8')
    
def create_data_mnist(path):
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    return X, y, X_test, y_test

In [7]:
X, y, X_test, y_test = create_data_mnist('fashion_mnist_images')

keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

X = (X.reshape(X.shape[0], -1).astype(np.float32) - 127.5) / 127.5
X_test = (X_test.reshape(X_test.shape[0], -1).astype(np.float32) - 127.5) / 127.5

model = ann.Model()

model.add(ann.Layer_Dense(X.shape[1], 128))
model.add(ann.Activation_ReLU())
model.add(ann.Layer_Dense(128, 128))
model.add(ann.Activation_ReLU())
model.add(ann.Layer_Dense(128, 10))
model.add(ann.Activation_Softmax())

model.set(
    loss=ann.Loss_CategoricalCrossentropy(),
    optimizer=ann.Optimizer_Adam(decay=1e-3),
    accuracy=ann.Accuracy_Categorical()
)

model.finalize()

model.train(X, y, validation_data=(X_test, y_test),epochs=10, batch_size=128, print_every=100)

epoch: 1
step: 0, acc: 0.086, loss: 2.303 (data_loss: 2.303, reg_loss: 0.000), lr: 0.001
step: 100, acc: 0.719, loss: 0.807 (data_loss: 0.807, reg_loss: 0.000), lr: 0.0009090909090909091
step: 200, acc: 0.703, loss: 0.688 (data_loss: 0.688, reg_loss: 0.000), lr: 0.0008333333333333334
step: 300, acc: 0.828, loss: 0.463 (data_loss: 0.463, reg_loss: 0.000), lr: 0.0007692307692307692
step: 400, acc: 0.789, loss: 0.512 (data_loss: 0.512, reg_loss: 0.000), lr: 0.0007142857142857144
step: 468, acc: 0.823, loss: 0.461 (data_loss: 0.461, reg_loss: 0.000), lr: 0.0006811989100817438
training, acc: 0.755, loss: 0.655 (data_loss: 0.655, reg_loss: 0.000), lr: 0.0006811989100817438
validation, acc: 0.824, loss: 0.489
epoch: 2
step: 0, acc: 0.812, loss: 0.543 (data_loss: 0.543, reg_loss: 0.000), lr: 0.0006807351940095302
step: 100, acc: 0.844, loss: 0.498 (data_loss: 0.498, reg_loss: 0.000), lr: 0.0006373486297004462
step: 200, acc: 0.773, loss: 0.500 (data_loss: 0.500, reg_loss: 0.000), lr: 0.0005991

In [8]:
model.evaluate(X_test, y_test)

validation, acc: 0.870, loss: 0.361


In [9]:
model.save('fashion_mnist.model')